# ETL pipeine
This notebook:
* Extracts amazon fashion reviews from HuggingFace dataset
* Transforms the data for downstream taks
* Loads intermediate results and final results into CSV-files 

In [1]:
# library import 

import re
from zipfile import ZipFile
import pandas as pd 
from datasets import load_dataset
from langdetect import detect, DetectorFactory
from transformers import set_seed
from collections import Counter
from bs4 import BeautifulSoup
import emoji

# Set seeds for reproducible and consistent results
set_seed(42)

/home/tom/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Extract 

In [2]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Amazon_Fashion", trust_remote_code=True)
print(dataset["full"][0])

# Overview
reviews = dataset['full'].to_pandas()


# Check the first 5 observations
print('----------------------------------------\nThe first 5 observations of the dataset:\n----------------------------------------')
display(reviews.head())

# Check the number of observations and features
print('--------------------------\nDimensions of the dataset:\n--------------------------\n')
print('Observations: ' + str(reviews.shape[0]))
print('Variables: ' + str(reviews.shape[1]))
print('Unique Products: ' + str(len(reviews['asin'].unique())))
print('Non-verified purchases: ' + str(len(reviews[reviews['verified_purchase'] == False])))

#
print('\n--------------------------\nMissing data distribution:\n--------------------------\n')
print(reviews.isna().sum())


{'rating': 5.0, 'title': 'Pretty locket', 'text': 'I think this locket is really pretty. The inside back is a solid silver depression and the front is a dome that is not solid (knotted). You could use it to store a small photo, lock of hair, etc but I use it when I need to carry medication with me. Closes securely. High quality & very pretty.', 'images': [], 'asin': 'B00LOPVX74', 'parent_asin': 'B00LOPVX74', 'user_id': 'AGBFYI2DDIKXC5Y4FARTYDTQBMFQ', 'timestamp': 1578528394489, 'helpful_vote': 3, 'verified_purchase': True}
----------------------------------------
The first 5 observations of the dataset:
----------------------------------------


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Pretty locket,I think this locket is really pretty. The insi...,[],B00LOPVX74,B00LOPVX74,AGBFYI2DDIKXC5Y4FARTYDTQBMFQ,1578528394489,3,True
1,5.0,A,Great,[],B07B4JXK8D,B07B4JXK8D,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1608426246701,0,True
2,2.0,Two Stars,One of the stones fell out within the first 2 ...,[],B007ZSEQ4Q,B007ZSEQ4Q,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,1432344828000,3,True
3,1.0,Won’t buy again,Crappy socks. Money wasted. Bought to wear wit...,[],B07F2BTFS9,B07F2BTFS9,AFVNEEPDEIH5SPUN5BWC6NKL3WNQ,1546289847095,2,True
4,5.0,I LOVE these glasses,I LOVE these glasses! They fit perfectly over...,[],B00PKRFU4O,B00XESJTDE,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1439476166000,0,True


--------------------------
Dimensions of the dataset:
--------------------------

Observations: 2500939
Variables: 10
Unique Products: 874297
Non-verified purchases: 163237

--------------------------
Missing data distribution:
--------------------------

rating               0
title                0
text                 0
images               0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64


## Transform


In [ ]:
raw_number_of_reviews = len(reviews) # Keep track of initial # of rows

### Data Cleansing
* Removing empty reviews
* Removing duplicates
* Removing non-verified purchases
* Removing reviews from bots
* Removing non-english reviews


In [ ]:
# Remove rows with empty text string
reviews = reviews[reviews['text'] != '']
diff = raw_number_of_reviews - len(reviews)
print(f'Removed {diff} rows')

# Remove duplicate text
reviews = reviews.drop_duplicates(subset=['text'], keep='first').reset_index(drop=True)
diff = raw_number_of_reviews - len(reviews)
print(f'Removed {diff} rows')

# Remove non-verified purchases
reviews = reviews[reviews['verified_purchase'] == True]
diff = raw_number_of_reviews - len(reviews)
print(f'Removed {diff} rows')

#### Identify bot behavior

In [3]:
# Check for bot_like behavior
reviews['datetime'] = pd.to_datetime(reviews['timestamp'], unit='ms')
reviews['date'] = reviews['datetime'].dt.strftime('%Y-%m-%d')

# Group_by to identify how manny reviews users make per day
review_counts = reviews[['user_id', 'date','text']].groupby(['user_id', 'date']).count()
review_counts.reset_index(inplace=True)
review_counts.rename(columns={'text': 'count'}, inplace=True)
review_counts.sort_values('count', ascending=False, inplace=True)
review_counts.head(10)

,user_id,date,count
339362,AEM5PO3PITEPDNNATSGM2PD6C62Q,2018-06-29,18
1030514,AFR2XB7TRMQJCRB6INYP2W73EH5A,2021-06-08,16
305453,AEKEF7PBXCP345X2UPV3IJY6QMNQ,2017-01-06,13
1022576,AFQNKOJVCTSPUXQIHHKIZHUC4U2Q,2014-12-04,13
2004685,AHF7II6UR3KZ7456G4LVM5K4JVQQ,2019-01-04,13
896044,AFJWIAJZOMSJFKQQDMOXUEJZKDVQ,2018-07-10,12
1582198,AGON3GV6Q37EUUSTQUWEAZ35D5JA,2017-04-15,12
1148317,AFXF3EGQTQDXMRLDWFU7UBFQZB7Q,2020-11-29,11
443634,AERPMJNZRD76NEI6NJBPRXKKEPMQ,2022-05-16,11
2289655,AHUEULPD67THST4AIUTZFI7GQAEQ,2017-11-17,10


In [4]:
# Get an understanding of commenting behaviors to find a proper threshold
for i in range(0,20,1):
    unique_users= len(review_counts[review_counts['count']>i]['user_id'].unique())
    print(f'{unique_users} users have posted more than {i} reviews on at least one day')

2035490 users have posted more than 0 reviews on at least one day
83488 users have posted more than 1 reviews on at least one day
10351 users have posted more than 2 reviews on at least one day
2907 users have posted more than 3 reviews on at least one day
872 users have posted more than 4 reviews on at least one day
379 users have posted more than 5 reviews on at least one day
148 users have posted more than 6 reviews on at least one day
71 users have posted more than 7 reviews on at least one day
26 users have posted more than 8 reviews on at least one day
15 users have posted more than 9 reviews on at least one day
9 users have posted more than 10 reviews on at least one day
7 users have posted more than 11 reviews on at least one day
5 users have posted more than 12 reviews on at least one day
2 users have posted more than 13 reviews on at least one day
2 users have posted more than 14 reviews on at least one day
2 users have posted more than 15 reviews on at least one day
1 users 

In [ ]:
# Set threshhold for bot_behavior to 3 or more reviews on one day
threshold = 3
bot_users = review_counts[review_counts['count']>=threshold]
bot_users_id_list = bot_users['user_id'].unique().tolist()


In [ ]:
# Remove bot like users
reviews = reviews[~reviews['user_id'].isin(bot_users_id_list)]
diff = raw_number_of_reviews - len(reviews)
print(f'Removed {diff} rows')


#### Remove non-english content

In [6]:
# Setting a consistent seed
DetectorFactory.seed = 12345

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown/not recognizable'

# Apply the function to detect language
reviews['language'] = reviews['text'].apply(detect_language)

# Count the frequency of each language
language_counts = Counter(reviews['language'])
display(language_counts)


Counter({'en': 1993151,
         'es': 21025,
         'ro': 8243,
         'af': 7697,
         'fr': 4207,
         'ca': 4125,
         'no': 3962,
         'de': 3564,
         'da': 3451,
         'it': 2651,
         'et': 2601,
         'so': 2408,
         'nl': 2360,
         'tl': 1623,
         'cy': 1483,
         'sl': 1427,
         'pt': 1341,
         'sv': 995,
         'sq': 979,
         'vi': 928,
         'pl': 726,
         'unknown/not recognizable': 710,
         'id': 684,
         'cs': 645,
         'fi': 444,
         'hr': 431,
         'sk': 411,
         'hu': 333,
         'tr': 308,
         'sw': 264,
         'lv': 117,
         'lt': 114,
         'bg': 1,
         'he': 1,
         'ko': 1,
         'zh-cn': 1})

In [7]:
# Remove non-english reviews
reviews = reviews[reviews['language'] == 'en']
diff = raw_number_of_reviews - len(reviews)
print(f'Removed {diff} rows')


Removed 80261 rows


In [ ]:
print('Number of reviews before data cleansing ' + str(raw_number_of_reviews))
print('Number of reviews after data cleansing: ' + str(len(reviews)))

## Data Cleaning
* Identifying flaws in the reviews
* Removing flaw patterns from text column

In [9]:
# Manual checking
for i in range(1000):
    print(f'{i}: {reviews["text"].to_list()[i]}')

0: I think this locket is really pretty. The inside back is a solid silver depression and the front is a dome that is not solid (knotted). You could use it to store a small photo, lock of hair, etc but I use it when I need to carry medication with me. Closes securely. High quality & very pretty.
1: Great
2: One of the stones fell out within the first 2 weeks of wearing it. Stones smaller than expected.
3: Crappy socks. Money wasted. Bought to wear with my tieks. Don’t stay on feet well.
4: I LOVE these glasses!  They fit perfectly over my regular, rectangular glasses that I always have to wear in order to see.  I really appreciate having these pretty and stylish and sturdy sunglasses to wear over my glasses.  I'll buy these again and again whenever I need a new pair, which hopefully won't be too soon.
5: Works OK for adults and is not at all kid friendly as stated. Slides down my 5 year olds face.
6: Great fit
7: Would be five stars but they are just too heavy for the size. So cute, ge

In [15]:
sample = reviews.iloc[:1000].copy()

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML
    text = emoji.replace_emoji(text, '') # Emojis
    text = re.sub(r'  ', ' ',text) # Double whitespaces
    return text

sample['cleaned'] = sample['text'].apply(clean_text)

C:\Users\Tom\AppData\Local\Temp\ipykernel_19984\799347660.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text #HTML


In [13]:
# Manual checking cleaned text column
for i in range(1000):
    print(f'{i}: {sample["text_cleaned"].to_list()[i]}')

0: I think this locket is really pretty. The inside back is a solid silver depression and the front is a dome that is not solid (knotted). You could use it to store a small photo, lock of hair, etc but I use it when I need to carry medication with me. Closes securely. High quality & very pretty.
1: Great
2: One of the stones fell out within the first 2 weeks of wearing it. Stones smaller than expected.
3: Crappy socks. Money wasted. Bought to wear with my tieks. Don’t stay on feet well.
4: I LOVE these glasses! They fit perfectly over my regular, rectangular glasses that I always have to wear in order to see. I really appreciate having these pretty and stylish and sturdy sunglasses to wear over my glasses. I'll buy these again and again whenever I need a new pair, which hopefully won't be too soon.
5: Works OK for adults and is not at all kid friendly as stated. Slides down my 5 year olds face.
6: Great fit
7: Would be five stars but they are just too heavy for the size. So cute, gets 

In [16]:
#  Apply to the whole dataset
reviews['text'] = reviews['text'].apply(clean_text)


C:\Users\Tom\AppData\Local\Temp\ipykernel_19984\799347660.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text #HTML
C:\Users\Tom\AppData\Local\Temp\ipykernel_19984\799347660.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text #HTML


### Load intermediate results into compressed csv
--> Checkpoint1

In [17]:
# Save the DataFrame to a CSV and compress to ZipFile
with ZipFile('Data/checkpoint1.zip', 'w') as z:
    with z.open('checkpoint1.csv', 'w') as f:
        reviews.to_csv(f, index=False)


## Data Enrichment
Adding sentiment labels for the text content to the data:
* Enriching data in batches as data is large
* Distributing inference to kaggle as GPUs are not locally available

In [8]:
# Split data in btaches to avoid runtime errors in kaggle
split = int(len(reviews)/2)
print(split)
reviews1 = reviews.iloc[:split]
reviews2 = reviews.iloc[split:]

996575


In [9]:
# This data is not included in repository to save space and is internally distributed via cloud
split = int(len(reviews1)/2)
print(split)
reviews11 = reviews1.iloc[:split]
reviews11.to_csv('senti_split11.csv', index = False)
reviews12 = reviews1.iloc[split:]
reviews12.to_csv('senti_split12.csv', index = False)

split = int(len(reviews2)/2)
print(split)
reviews21 = reviews2.iloc[:split]
reviews21.to_csv('senti_split21.csv', index = False)
reviews22 = reviews2.iloc[split:]
reviews22.to_csv('senti_split22.csv', index = False)


498287
498288


In [ ]:
# This part is run on kaggle for each batch
# Checkout sentiment.ipynb

import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm  

import numpy as np 
import pandas as pd 
from transformers import AutoModelForSequenceClassification, AutoTokenizer, set_seed


# Set random seeds for reproducible and consistent results
set_seed(42)

# Change paths here
input = "kaggle_environment_input_path"
output = "senti_split_11"

df = pd.read_csv(input)
checkpoint = 'siebert/sentiment-roberta-large-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

class ScamDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=self.max_length)
        return inputs

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
model.eval()  # Set the model to evaluation mode

# Create a Dataset and DataLoader
texts = df['text'].copy()
dataset = ScamDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

predictions = []

with torch.no_grad():
    # Wrap the dataloader with tqdm to track progress
    for batch in tqdm(dataloader, desc="Classifying"):
        # Move batch data to GPU
        inputs = {key: val.squeeze(1).to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1).tolist()
        predictions.extend(batch_predictions)

print("Classification complete.")

# Add predictions to DataFrame
df['label'] = predictions
df.to_csv(output)

### Load intermediate results into compressed csv
--> Checkpoint2

In [2]:
# with ZipFile('Data/checkpoint1.zip', 'r') as zip:
#         with zip.open('checkpoint1.csv') as file:
#                 reviews = pd.read_csv(file)

In [3]:
sentiments = pd.DataFrame()

for split in [11,12,21,22]:

    with ZipFile(f'Data/senti_split_{split}.zip', 'r') as zip:
        with zip.open(f'senti_split_{split}.csv') as file:
            senti_split = pd.read_csv(file)
            sentiments = pd.concat([sentiments,senti_split])

# Save the DataFrame to a CSV and compress to ZipFile
with ZipFile('Data/checkpoint2.zip', 'w') as z:
    with z.open('checkpoint2.csv', 'w') as f:
        sentiments.to_csv(f, index=False)



In [4]:
# Check integrety of data
len(sentiments) == len(reviews)

False

### Load training data into csv-file

--> checkpoint3

In [9]:
# Split pos/neg 40/60 to avoid positivity bias in gpt2 as training data
df_pos = sentiments[sentiments['label'] == 1].sample(40000) # 1 = positive
df_neg = sentiments[sentiments['label'] == 0].sample(60000) # 0 = negative
df_sampled = pd.concat([df_pos, df_neg])

# Save the DataFrame to a CSV and compress to ZipFile
with ZipFile('Data/checkpoint3.zip', 'w') as z:
    with z.open('checkpoint3.csv', 'w') as f:
        df_sampled.to_csv(f, index=False)



In [ ]:
# Check integrety
print(df_sampled['label'].value_counts()) # 1 = positive; # 0 = negative